In [2]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data_path = os.path.join('..', 'data' ,'raw', 'train.json')
f = open(data_path)
data = json.load(f)
f.close()


In [91]:
test_data_path = os.path.join('..', 'data' ,'raw', 'test.json')
f = open(test_data_path)
test_data = json.load(f)
f.close()


In [4]:
data_df = pd.DataFrame(data)
data_df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [93]:
test_df = pd.DataFrame(test_data)
test_df.head()

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."


In [5]:
all_labels = set([label for sublist in data_df['labels'] for label in sublist])
all_labels
label_counts = (data_df['labels'].apply(pd.Series).stack().value_counts())
label_counts

O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

In [110]:
import re 
# [a-zA-Z0-9]{0,}([.]?[a-zA-Z0-9]{1,})[@](gmail.com|hotmail.com|yahoo.com)

EMAIL_REGEX = r'\b[A-Za-z0-9]+[.-_]*[A-Za-z0-9]+@(gmail|hotmail|yahoo)+\.[A-Z|a-z]{2,}\b'
# EMAIL_REGEX = r'\b[A-Za-z0-9]+[.-_]*[A-Za-z0-9]+@[A-Za-z0-9-]+\.[A-Z|a-z]{2,}\b'
PHONE_NUMEBR_REGEX = r'\(\d{1,3}\)\d{3}-\d+(?:x\d+)?|\d{3}\.\d{3}\.\d{1,10}'
# PHONE_NUMEBR_REGEX = r'\(\d{1,3}\)\d{3,14}-\d+(?:x\d+)?|\d{3}\.\d{1,4}\.\d{1,10}'
ADDRESS_REGEX = r'^\d+ [a-zA-Z\s]+(?:\. \d+)[a-zA-Z\s]+(?:\, [A-Z]{2}\s\d{5,6})$'


PHONE_SECOND_PART_REGEX = r'\b\d{1,3}\)\d{1,4}\b'
PHONE_FOURTH_PART_REGEX =  r'\d+(x\d*)?'
NUMBER_REGEX = r'\d+'


def find_email(text):
    return re.findall(re.compile(EMAIL_REGEX), text)

def find_phone_number(text):
    return re.findall(re.compile(PHONE_NUMEBR_REGEX), text)

def find_address(text):
    return re.findall(re.compile(ADDRESS_REGEX), text)


def check_number(token):
    return re.fullmatch(re.compile(NUMBER_REGEX), token)

def check_partial_phone_number(token, second_token, third_token, fourth_token):
    if token == "(" and check_phone_second_part(second_token) and third_token == "-" and check_phone_fourth_part(fourth_token):
        return True
    return False

def check_phone_second_part(token):
    return re.fullmatch(re.compile(PHONE_SECOND_PART_REGEX), token)

def check_phone_fourth_part(token):
    return re.fullmatch(re.compile(PHONE_FOURTH_PART_REGEX), token)

In [7]:
filtered_rows = data_df[data_df['labels'].apply(lambda x: 'I-STREET_ADDRESS' in x)]
filtered_rows

,document,full_text,tokens,trailing_whitespace,labels
1103,9854,Waseem Mabunda 591 Smith Centers Apt. 656\nJo...,"[Waseem, Mabunda, , 591, Smith, Centers, Apt,...","[True, True, False, True, True, True, False, T...","[B-NAME_STUDENT, I-NAME_STUDENT, O, B-STREET_A..."
1887,11442,Reflection – Learning Launch of 1861. Milano\n...,"[Reflection, –, Learning, Launch, of, 1861, .,...","[True, True, True, True, True, False, True, Fa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [115]:
# filtered_rows = data_df[data_df['labels'].apply(lambda x: 'B-STREET_ADDRESS' in x)]
filtered_rows = data_df[data_df['labels'].apply(lambda x: 'B-EMAIL' in x)]['tokens']
# res  = filtered_rows['full_text'].apply(find_phone_number)
# print(filtered_rows['full_text'][1103])
# print(filtered_rows['labels'][1103])
# filtered_rows['tokens'][1103]
# filtered_rows

s = "vpi@mn.nl"
print(find_email(s))
# filtered_rows

[]


Filter tokens whose tag contains STREET_ADDRESS

In [98]:
addresses = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if '-STREET_ADDRESS' in label]
addresses
s = str.join(' ', ['591', 'Smith','Centers','Apt.','656','\n','Joshuamouth,','RI','95963'])
s2 = str.join(' ', ['743', 'Erika','Bypass','Apt.','419','\n','Andreahaven,','IL','54207'])
print(s)
# print(s2)

# pattern = r'^(\d{1,}) [a-zA-Z\s]+(\.)?\s[A-Z]{2}\s\d{5,6}$'
pattern = r'^\d+ [a-zA-Z\s]+(?:\. \d+)[a-zA-Z\s]+(?:\, [A-Z]{2}\s\d{5,6})$'
match = re.findall(pattern, '591 Smith Centers Apt. 656\nJoshuamouth, RI 95963')
print(match)

591 Smith Centers Apt. 656 
 Joshuamouth, RI 95963
['591 Smith Centers Apt. 656\nJoshuamouth, RI 95963']


Filter tokens whose tag is B-EMAIL

In [114]:
emails = [token for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if label == 'B-EMAIL']
emails

['djones@gmail.com',
 'matthew72@hotmail.com',
 'belindarojas@yahoo.com',
 'kennethevans@hotmail.com',
 'agood@gmail.com',
 'agood@gmail.com',
 'hwillis@gmail.com',
 'kellyharrison@gmail.com',
 'kellyharrison@gmail.com',
 'lowetyler@hotmail.com',
 'murraythomas@gmail.com',
 'vmartinez@hotmail.com',
 'sheena82@yahoo.com',
 'nbarker@hotmail.com',
 'nbarker@hotmail.com',
 'nbarker@hotmail.com',
 'johnsondavid@hotmail.com',
 'johnsondavid@hotmail.com',
 'catherine19@hotmail.com',
 'john03@yahoo.com',
 'john03@yahoo.com',
 'lisarose@gmail.com',
 'diazkristen@gmail.com',
 'lisarose@gmail.com',
 'diazkristen@gmail.com',
 'andrew58@gmail.com',
 'boydcynthia@yahoo.com',
 'laurie33@yahoo.com',
 'laurie33@yahoo.com',
 'pauljohnson@hotmail.com',
 'pauljohnson@hotmail.com',
 'jarviscindy@hotmail.com',
 'jarviscindy@hotmail.com',
 'jray@hotmail.com',
 'gday@hotmail.com',
 'gday@hotmail.com',
 'gday@hotmail.com',
 'noah14@hotmail.com',
 'hbrown@yahoo.com']

Filter tokens whose tag contains PHONE_NUM

In [230]:
phones = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if '-PHONE_NUM' in label]
phones

[('(', 'B-PHONE_NUM'),
 ('320)202', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('0688x95843', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('223)392', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('2765', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('410.526.1667', 'B-PHONE_NUM')]

Check '(' token label

In [89]:
paranthesis_tokens = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if token=='(']
paranthesis_labels = {item for sublist in paranthesis_tokens for item in sublist if item != '('}
paranthesis_labels

{'B-PHONE_NUM', 'O'}

Chech '-' token label

In [90]:
hyphen_tokens = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if token=='-']
hyphen_labels = {item for sublist in hyphen_tokens for item in sublist if item != '-'}
hyphen_labels

{'I-PHONE_NUM', 'O'}

Filter number tokens with their labels

In [186]:
numbers = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if check_number(token)]
n  = {item for sublist in numbers for item in sublist if not check_number(item)}
n

{'B-ID_NUM',
 'B-STREET_ADDRESS',
 'I-ID_NUM',
 'I-PHONE_NUM',
 'I-STREET_ADDRESS',
 'O'}